## Final Project Submission

Please fill out:
* Student name: Michael Mahoney
* Student pace:  part time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


General notes:
look into movies that are less expensive. This might be a way to limit capital risk if Microsoft doesn't want to go super hard into the investment. 
look at distribution of the top grossing files. Maybe give them a distribution of files to go for, show which ones will have pretty reliable income and which ones are the longshots but have huge payoff potential. 

imports os and glob
use pd.merge

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import seaborn as sns

In [2]:
dfBomGross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
dfimdbName = pd.read_csv('zippedData/imdb.name.basics.csv.gz')

dfimdbTitleAkas = pd.read_csv('zippedData/imdb.title.akas.csv.gz')
columns = list(dfimdbTitleAkas.columns)
columns[0] = 'tconst'
dfimdbTitleAkas.columns = columns

dfimdbTitleBasics = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
dfimdbTitleCrew = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
dfimdbTitlePrincipals = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
dfimdbTitleRatings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
dfRtMovie = pd.read_csv('zippedData/rt.movie_info.tsv.gz',sep='\t')
dfRtReviews = pd.read_csv('zippedData/rt.reviews.tsv.gz',sep='\t', encoding='latin_1')
dfTmbd = pd.read_csv('zippedData/tmdb.movies.csv.gz')
dfTn = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

listOfImdbDfs = [dfimdbName,dfimdbTitleAkas,dfimdbTitleBasics,dfimdbTitleCrew,dfimdbTitlePrincipals,dfimdbTitleRatings]
for x in listOfImdbDfs:
    if('tconst' in x.keys()):
        x['tconst'] = x['tconst'].astype('str')

In [3]:
display(dfBomGross.head(1),
dfimdbName.head(1),
dfimdbTitleAkas.head(1),dfimdbTitleAkas['tconst'][:5],
dfimdbTitleBasics.head(1),
dfimdbTitleCrew.head(1),
dfimdbTitlePrincipals.head(1),
dfimdbTitleRatings.head(1),
dfRtMovie.head(1),
dfRtReviews.head(1),
dfTmbd.head(1),
dfTn.head(1))



,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"


,tconst,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0


0    tt0369610
1    tt0369610
2    tt0369610
3    tt0369610
4    tt0369610
Name: tconst, dtype: object

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"


...

,tconst,averagerating,numvotes
0,tt10356526,8.3,31


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN


,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


## Looking at this structure, it's clear that joining tables with only financial information isn't going to give much insight as to why they perform the way they do. For the sake of saving time later, I'm going to combine all of the IMDB info here and use the names of the movies we do have income information on to sort through the IMDB list which is much more complete in terms of information.

In [110]:
dfimdbTitleAkas['tconst'] = dfimdbTitleAkas['tconst'].astype('str')
dfimdbTitleBasics['tconst'] = dfimdbTitleBasics['tconst'].astype('str')
dfimdbTitleCrew['tconst'] = dfimdbTitleCrew['tconst'].astype('str')
dfimdbTitlePrincipals['tconst'] = dfimdbTitlePrincipals['tconst'].astype('str')
dfimdbTitleRatings['tconst'] = dfimdbTitleRatings['tconst'].astype('str')
imdbList = [dfimdbTitleAkas,dfimdbTitleBasics,dfimdbTitleCrew,dfimdbTitlePrincipals,dfimdbTitleRatings]

def joinThings(df1, df2):
    df = df1.merge(df2,how='left',left_on='tconst', right_on='tconst')
    return df

df = joinThings(joinThings(joinThings(joinThings(dfimdbTitleAkas,dfimdbTitleBasics),dfimdbTitleCrew),dfimdbTitlePrincipals),dfimdbTitleRatings)

# # df = dfimdbTitleAkas.merge(dfimdbTitleBasics,how='left',left_on='tconst', right_on='tconst')
# df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2841276 entries, 0 to 2841275
Data columns (total 22 columns):
 #   Column             Dtype  
---  ------             -----  
 0   tconst             object 
 1   ordering_x         int64  
 2   title              object 
 3   region             object 
 4   language           object 
 5   types              object 
 6   attributes         object 
 7   is_original_title  float64
 8   primary_title      object 
 9   original_title     object 
 10  start_year         int64  
 11  runtime_minutes    float64
 12  genres             object 
 13  directors          object 
 14  writers            object 
 15  ordering_y         float64
 16  nconst             object 
 17  category           object 
 18  job                object 
 19  characters         object 
 20  averagerating      float64
 21  numvotes           float64
dtypes: float64(5), int64(2), object(15)
memory usage: 498.6+ MB


In [141]:
df2 = dfTn.merge(dfBomGross, how = 'left', left_on='movie', right_on = 'title')
listOfNames = [x for x in df2.movie.unique()]
def test(x):
    if x in listOfNames:
        return True
    else:
        return False
df['title'].loc[lambda x: test(x)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [100]:

# convert object data to int data for analysis
df = dfTn.merge(dfBomGross, how = 'left', left_on='movie', right_on = 'title' )

df['domestic_gross_x'] = [x.replace(',','').replace('$','') for x in df['domestic_gross_x']]
df['domestic_gross_x'] = df['domestic_gross_x'].astype('int64')
df['worldwide_gross'] = [x.replace(',','').replace('$','') for x in df['worldwide_gross']]
df['worldwide_gross'] =df['worldwide_gross'].astype('int64')
df['total_gross'] = df['domestic_gross_x']+df['worldwide_gross']



In [101]:
df.sort_values(['domestic_gross_x'],axis=0, ascending=False)






# display(dfBomGross.loc[dfBomGross['foreign_gross'].isna()])
# dfBomGross['foreign_gross'] = dfBomGross['foreign_gross'].astype('int')
# dfBomGross['total_gross'] = dfBomGross['domestic_gross']+dfBomGross['foreign_gross']

,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,title,studio,domestic_gross_y,foreign_gross,year,total_gross
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000",936662225,2053311220,NaN,NaN,NaN,NaN,NaN,2989973445
0,1,"Dec 18, 2009",Avatar,"$425,000,000",760507625,2776345279,NaN,NaN,NaN,NaN,NaN,3536852904
41,42,"Feb 16, 2018",Black Panther,"$200,000,000",700059566,1348258224,Black Panther,BV,700100000.0,646900000,2018.0,2048317790
6,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000",678815482,2048134200,Avengers: Infinity War,BV,678800000.0,"1,369.5",2018.0,2726949682
42,43,"Dec 19, 1997",Titanic,"$200,000,000",659363944,2208208395,NaN,NaN,NaN,NaN,NaN,2867572339
...,...,...,...,...,...,...,...,...,...,...,...,...
2709,10,"Mar 31, 2004",The Touch,"$20,000,000",0,5918742,NaN,NaN,NaN,NaN,NaN,5918742
2708,9,"Apr 13, 2010",Three Kingdoms: Resurrection of the Dragon,"$20,000,000",0,22139590,NaN,NaN,NaN,NaN,NaN,22139590
2707,8,"Dec 31, 2012",Zambezia,"$20,000,000",0,34454336,NaN,NaN,NaN,NaN,NaN,34454336
2706,7,"Dec 31, 2008",Admiral,"$20,000,000",0,38585047,NaN,NaN,NaN,NaN,NaN,38585047


dtype('O')